# Categorizing urban density

last modified: February, 2016

author: [Raj Singh](https://developer.ibm.com/clouddataservices/author/rrsingh/)

original: https://github.com/ibm-cds-labs/open-data/blob/master/samples/urbanity.ipynb


## Overview

Exploration of an academic study of urban structure and density described in the June 2014 article, ["From Jurisdictional to Functional Analysis of Urban Cores & Suburbs"](http://www.newgeography.com/content/004349-from-jurisdictional-functional-analysis-urban-cores-suburbs) in [new geography](http://www.newgeography.com/). 

## Categories

- Urban (pre-auto urban core): density > 2,900 sq. km
- Auto suburban, early: median house built 1946 to 1979, density < 2,900 sq. km and density > 100 sq. km
- Auto suburban, later: median house built after 1979, density < 2,900 sq. km and density > 100 sq. km
- Auto exurban: all others


In [4]:
import pandas as pd, numpy as np, os

## Collect U.S. Census data

Census data from the 2013 US Census American Community Survey (ACS), 5-year estimates.

Created from the "zip code tabulation area" (ZCTA) [TIGER/Line® with Selected Demographic and Economic Data product in Geodatabase format](http://www.census.gov/geo/maps-data/data/tiger-data.html). This particular version of the ACS is used for the folowing reasons:

1. 5-year estimates are the most accurate data outside of the decennial census [as explained here](http://www.census.gov/programs-surveys/acs/guidance/estimates.html).
1. 2013 is the most recent data set with 5-year estimates
1. TIGER/Line® gives you the geographic boundaries of the zip codes so you can perform spatial analyses
1. This data set is smaller than the full Census, but still has the important income, education, race, age and occupation demographics we want to use.

If you want to do this yourself, [this article](https://developer.ibm.com/clouddataservices/2015/09/08/census-open-data-on-ibm-cloud/) explains how to get a CSV out of that format.

### Accessing Census files

Access the files mentioned here via the [IBM Analytics Exchange](https://console.ng.bluemix.net/data/exchange/). You can get your free, individual "API key" on each data set's detail page:
 
- [Zip code areas](https://console.ng.bluemix.net/data/exchange/public/e/united-states-demographic-measures-land)
- [Age](https://console.ng.bluemix.net/data/exchange/public/e/united-states-demographic-measures-age)
- [Housing](https://console.ng.bluemix.net/data/exchange/public/e/united-states-demographic-measures-housing)

After getting your API key (your personal URL for the data file), in this script replace, for example, `os.environ['AE_KEY_AGE']` with your URL for the Age data CSV file.


### Get zip code areas from Census

In [5]:
geo_df = pd.read_csv( os.environ['AE_KEY_LAND'], usecols=['GEOID','ALAND10'], dtype={"GEOID10": np.str} )
geo_df = geo_df.set_index('GEOID')
geo_df.head()

,ALAND10
GEOID,
86000US43451,63411475
86000US43452,121783680
86000US43456,9389360
86000US43457,48035540
86000US43458,2573816


### Get population from Census

In [10]:
pop_df = pd.read_csv( os.environ['AE_KEY_AGE'], usecols=['GEOID','B01001e1'], dtype={"GEOID": np.str} )
pop_df.columns = ['GEOID','POPULATION']
pop_df = pop_df.set_index('GEOID')
pop_df.head()

,POPULATION
GEOID,
86000US01001,17245
86000US01002,29266
86000US01003,11032
86000US01005,5356
86000US01007,14673


### Get housing age from Census
NOTE: this is a large 210Mb file and may take a few minutes to load

In [16]:
housing_df = pd.read_csv( os.environ['AE_KEY_HOUSING'], usecols=['GEOID','B25035e1'], dtype={"GEOID": np.str} )
housing_df = housing_df.set_index('GEOID')
housing_df.sample(5)

,B25035e1
GEOID,
86000US80239,1985.0
86000US95658,1977.0
86000US50248,1972.0
86000US66053,1991.0
86000US20565,NaN


### Join Census data into one DataFrame with nice names

In [17]:
urban_df = geo_df.join(pop_df)
urban_df = urban_df.join(housing_df)
urban_df.sample(5)

,ALAND10,POPULATION,B25035e1
GEOID,,,
86000US48105,129210899,33729,1978.0
86000US16154,67464645,2780,1973.0
86000US86426,93845807,13902,1998.0
86000US53503,146106994,1894,1980.0
86000US24604,10567313,643,1971.0


In [18]:
urban_df.columns = ['AREAMSQ','Population','MEDYRBUILT']
urban_df.sample(5)

,AREAMSQ,Population,MEDYRBUILT
GEOID,,,
86000US95005,30929061,6168,1965.0
86000US32827,60494466,6576,NaN
86000US87543,52008262,535,1939.0
86000US59851,558857,254,1939.0
86000US48133,69548569,5560,1972.0


## Density calculation

- persons per square km = persons / (area in square meters / 1,000,000)
- persons per hectare = persons / (area in square meters / 10,000)

### Compute population density as persons per square kilometer

In [19]:
urban_df['POPPERKMSQ'] = urban_df['Population'] / (urban_df['AREAMSQ']/1000000)
urban_df.sample(4)

,AREAMSQ,Population,MEDYRBUILT,POPPERKMSQ
GEOID,,,,
86000US35016,192558438,17245,1982.0,89.557228
86000US96137,125380322,3826,1979.0,30.515155
86000US38343,358833001,15792,1972.0,44.009330
86000US67646,444281421,630,1951.0,1.418020


### Group population density into 4 categories

In [20]:
urban_df['CAT'] = 'EXURBAN'
urban_df['CAT'][(urban_df['POPPERKMSQ'] >= 2900)] = 'URBAN'
urban_df['CAT'][(urban_df['POPPERKMSQ'] < 2900) & (urban_df['POPPERKMSQ'] >= 100) & (urban_df['MEDYRBUILT'] < 1980) & (urban_df['MEDYRBUILT'] >= 1946)] = 'SUBURBANEARLY'
urban_df['CAT'][(urban_df['POPPERKMSQ'] < 2900) & (urban_df['POPPERKMSQ'] >= 100) & (urban_df['MEDYRBUILT'] >= 1980)] = 'SUBURBANLATE'
urban_df.describe()

/Users/rajrsingh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/rajrsingh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/rajrsingh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/rajrsingh/anaconda/lib/python2.7/site-package

,AREAMSQ,Population,MEDYRBUILT,POPPERKMSQ
count,3.298900e+04,32989.000000,32045.000000,32989.000000
mean,2.250019e+08,9443.177453,1971.068529,487.703913
std,6.575933e+08,13858.010530,15.606758,1912.093435
min,5.094000e+03,0.000000,1939.000000,0.000000
25%,2.351832e+07,719.000000,NaN,7.754449
50%,9.322068e+07,2781.000000,NaN,30.194351
75%,2.304373e+08,12830.000000,NaN,249.247358
max,3.478591e+10,114734.000000,2011.000000,71226.281507


In [21]:
# look at a few records to do a quick sanity check
urban_df.sample(30)

,AREAMSQ,Population,MEDYRBUILT,POPPERKMSQ,CAT
GEOID,,,,,
86000US43084,55869832,854,1947.0,15.285530,EXURBAN
86000US23480,5001165,273,1949.0,54.587281,EXURBAN
86000US44321,39417067,14933,1986.0,378.846047,SUBURBANLATE
86000US60551,149406633,4779,1982.0,31.986532,EXURBAN
86000US75103,516577037,13103,1981.0,25.365045,EXURBAN
86000US25414,141470974,17147,1986.0,121.205075,SUBURBANLATE
86000US37210,23936536,15071,1964.0,629.623267,SUBURBANEARLY
86000US11560,19430271,6534,1954.0,336.279406,SUBURBANEARLY
86000US49458,18569991,52,1974.0,2.800217,EXURBAN


## Mapping Urbanity

In [22]:
#!pip install --user cartodb
# Set up cartodb module for use
from cartodb import CartoDBAPIKey, CartoDBException
# Learn how to use the CartoDB SQL API here:
# http://docs.cartodb.com/cartodb-platform/sql-api/making-calls/
API_KEY = 'dfd11d7ad9ad3f213a6336b5e878865422f624f1'
CARTODB_ACCOUNT = 'ibm'
cl = CartoDBAPIKey(API_KEY, CARTODB_ACCOUNT)

### Update data table in CartoDB

In [23]:
# get all non-exurban zips
notex_df = urban_df[ urban_df['CAT'] != 'EXURBAN' ]
notex_df.head()

,AREAMSQ,Population,MEDYRBUILT,POPPERKMSQ,CAT
GEOID,,,,,
86000US43452,121783680,13659,1974.0,112.157885,SUBURBANEARLY
86000US43458,2573816,312,1960.0,121.220787,SUBURBANEARLY
86000US43460,7158543,6334,1959.0,884.816924,SUBURBANEARLY
86000US43465,28559486,5267,1974.0,184.422087,SUBURBANEARLY
86000US43468,1861424,403,1957.0,216.500915,SUBURBANEARLY


In [24]:
# generate PostgreSQL INSERT statements for CartoDB
insertsql = ''
for index, row in notex_df.iterrows(): 
    insertsql += "INSERT INTO urbanity(zipcode,category) VALUES ('" + index + "', '" + row['CAT'] + "');"

In [25]:
# send SQL INSERT statements to CartoDB
try:
    print(cl.sql('delete from urbanity'))
    print(cl.sql(insertsql))
except CartoDBException as e:
    print("some error ocurred", e)

{u'fields': {}, u'rows': [], u'total_rows': 10002, u'time': 0.059}
{u'fields': {}, u'rows': [], u'total_rows': 1, u'time': 55.475}


### Urbanity Map!

In [26]:
%%javascript
element.append("<link rel='stylesheet' href='http://libs.cartocdn.com/cartodb.js/v3/3.15/themes/css/cartodb.css' />")
element.append("<div id='map' style='height:500px;width:900px;padding:0;margin:0'></div>");

require.config({
  paths: {
      cartodblib: 'http://libs.cartocdn.com/cartodb.js/v3/3.15/cartodb'
  }
});

var main = function() {
  cartodb.createVis('map', 'https://ibmanalytics.cartodb.com/u/ibm/api/v2/viz/3a4e9e24-b3f6-11e5-a4c7-0e31c9be1b51/viz.json', {
      shareable: true,title: true,description: true,search: true,tiles_loader: true,
      center_lat: 40, center_lon: -100, zoom: 3
  })
  .done(function(vis, layers) {
    // layer 0 is the base layer, layer 1 is cartodb layer
    // setInteraction is disabled by default
    layers[1].setInteraction(true);
    layers[1].on('featureOver', function(e, latlng, pos, data) {
      cartodb.log.log(e, latlng, pos, data);
    });
    
    // var map = vis.getNativeMap(); // get the native map to work with it
    
  })
  .error(function(err) {
    console.log(err);
  });
}

require(['cartodblib'], main);

<IPython.core.display.Javascript object>